In [1066]:
#%pip uninstall ggplot
#%pip install pyshp

In [1]:
import pandas as pd
import numpy as np
import string
import re
import time
from bs4 import BeautifulSoup
from unicodedata import normalize

In [2]:
# FUNCTIONS ######################

In [3]:
industries = [
   'All', 'Food_Services', 'Waste_Management', 'Agriculture',
   'Arts', 'Construction', 'Education', 'Finance',
   'Healthcare', 'Information', 'Management', 'Manufacturing',
   'Mining', 'Other', 'Technical', 'Real_Estate', 'Retail',
   'Transportation', 'Utilities', 'Wholesale'
]

In [4]:
def convert_to_int(vec):
    return vec.astype("string").str.replace("%","",regex=False).str.replace("$", "", regex=False).str.replace(",", "", regex=False).astype("float").astype("Int32")

In [7]:
def get_table_df(html, table_label):
    soup = BeautifulSoup(html, 'html.parser')
    divs = soup.findAll('div', attrs = {'class':'MuiStack-root css-mxqgcz'})
    for d in divs:
        label = d["aria-label"]
        if(label == table_label):
            table = d.select_one("div[class='MuiStack-root css-mxqgcz'] table")
            try:
                return pd.read_html(str(table))[0]
            except:
                return None
    return None

In [8]:
def add_fields(df, fields, row):
    if df is None:
        for f in fields:
            row[f'{f}'] = 0
        return row
    else:
        index_df = df.set_index('Variable')
        for f in fields:
            row[f'{f}'] = float(str(index_df['Estimate'][f]).replace("%","").replace("$","").replace(",",""))
        return row

In [9]:
def get_business_totals(html):
    df = get_table_df(html, "Businesses Totals")
    
    if(df is None):
        return None
        
    cols = {
        'All Employer establishments (Total)': 'Employers', 
        'Total employment of employers':'Employees', 
        'Total annual payroll of employers ($1,000)':'Payroll', 
        'Total revenue of employers ($1,000)':'Revenue'
    }
    df.rename(columns=cols, inplace=True)
    
    # convert null values to zero
    df.loc[pd.isna(df.Employers), "Employers"] = '0'
    df.loc[pd.isna(df.Employees), "Employees"] = '0'
    df.loc[pd.isna(df.Payroll), "Payroll"] = '0'
    df.loc[pd.isna(df.Revenue), "Revenue"] = '0'

    # convert to int and strip chars ($,.)
    df.Employers = convert_to_int(df.Employers)
    df.Employees = convert_to_int(df.Employees)
    df.Payroll = convert_to_int(df.Payroll)
    df.Revenue = convert_to_int(df.Revenue)

    df = shorten_industries(df)
    
    return df

In [10]:
def add_business_fields_to_row(df, row):
    fields = ['Employers', 'Employees', 'Payroll', 'Revenue']    
    index_df = df.set_index('Industry')
    for i in industries:
        for f in fields:
            row[f'{i}_{f}'] = index_df[f][i]
    
    # recalculate totals since most in the survey are empty
    row['All_Employers'] = df['Employers'].sum()
    row['All_Employees'] = df['Employees'].sum()
    row['All_Payroll'] = df['Payroll'].sum()
    row['All_Revenue'] = df['Revenue'].sum()

    # calculate averages
    all_revenue = row['All_Revenue']
    all_employees = row['All_Employees']
    all_employers = row['All_Employers']
    row['Revenue_Per_Employee'] = 0 if all_employees == 0 else all_revenue / all_employees
    row['Revenue_Per_Employer'] = 0 if all_employers == 0 else all_revenue / all_employers
    row['Profit'] = all_revenue - row['All_Payroll']
    
    return row

In [11]:
def get_demographics(html):
    df = get_table_df(html, "Demographic Characteristics")
    
    if(df is None):
        return None
        
    df.loc[df.Variable == 'Total population','Variable'] = 'Population'
    df.loc[df.Variable == 'Percent male','Variable'] = 'Percent_Male'
    df.loc[df.Variable == 'Percent female','Variable'] = 'Percent_Female'
    df.loc[df.Variable == 'Percent under 5 years','Variable'] = 'Percent_Under_5'
    df.loc[df.Variable == 'Percent under 18 years','Variable'] = 'Percent_Under_18'
    df.loc[df.Variable == 'Percent 18 years and over','Variable'] = 'Percent_18_Over'
    df.loc[df.Variable == 'Percent 21 years and over','Variable'] = 'Percent_21_Over'
    df.loc[df.Variable == 'Percent working age (25 to 64 years)','Variable'] = 'Percent_25_64'
    df.loc[df.Variable == 'Percent 65 years and over','Variable'] = 'Percent_65_Over'
    df.loc[df.Variable == 'Median age','Variable'] = 'Median_Age'
    df.loc[df.Variable == 'Percent White','Variable'] = 'Percent_White'
    df.loc[df.Variable == 'Percent Black or African American','Variable'] = 'Percent_Black'
    df.loc[df.Variable == 'Percent American Indian and Alaska Native','Variable'] = 'Percent_Native'
    df.loc[df.Variable == 'Percent Asian','Variable'] = 'Percent_Asian'
    df.loc[df.Variable == 'Percent Native Hawaiian and Other Pacific Islander','Variable'] = 'Percent_Islander'
    df.loc[df.Variable == 'Percent some other race','Variable'] = 'Percent_Other'
    df.loc[df.Variable == 'Percent two or more races','Variable'] = 'Percent_Mixed'
    df.loc[df.Variable == 'Percent Hispanic (of any race)','Variable'] = 'Percent_Hispanic'
    
    df.replace(np.nan,0)
    
    return df

In [12]:
def add_demographics_fields_to_row(df, row):
    fields = ['Population', 'Percent_Male', 'Percent_Female', 'Percent_Under_5', 'Percent_Under_18', 'Percent_18_Over', 
              'Percent_21_Over', 'Percent_25_64', 'Percent_65_Over', 'Median_Age', 'Percent_White', 'Percent_Black', 
              'Percent_Native', 'Percent_Asian', 'Percent_Islander', 'Percent_Other', 'Percent_Mixed', 'Percent_Hispanic']    
    
    return add_fields(df, fields, row)

In [13]:
def get_socioeconomics(html):
    df = get_table_df(html, "Socioeconomic Characteristics")
    
    if(df is None):
        return None
        
    df.loc[df.Variable == 'Median household income','Variable'] = 'Median_Income'
    df.loc[df.Variable == 'Average income','Variable'] = 'Average_Income'
    df.loc[df.Variable == 'Percent high school degree or higher','Variable'] = 'Percent_High_School'
    df.loc[df.Variable == 'Percent Bachelor\'s degree or higher','Variable'] = 'Percent_Bachelors'
    df.loc[df.Variable == 'Average household size','Variable'] = 'Average_Household_Size'
    df.loc[df.Variable == 'Percent in poverty','Variable'] = 'Percent_In_Poverty'
    df.loc[df.Variable == 'Percent population 16 years and over in labor force','Variable'] = 'Percent_16_Over_Working'
    df.loc[df.Variable == 'Percent employed','Variable'] = 'Percent_Employed'
    df.loc[df.Variable == 'Percent disabled','Variable'] = 'Percent_Disabled'
    df.loc[df.Variable == 'Percent with health coverage','Variable'] = 'Percent_Health_Coverage'
    df.loc[df.Variable == 'Percent of all workers 16 and over who commute to work','Variable'] = 'Percent_16_Over_Commuters'
    df.loc[df.Variable == 'Percent who drive alone to work','Variable'] = 'Percent_Drive_Alone'
    df.loc[df.Variable == 'Percent who carpool to work','Variable'] = 'Percent_Drive_Carpool'
    df.loc[df.Variable == 'Percent who use public transportation to commute to work','Variable'] = 'Percent_Public_Transport'
    df.loc[df.Variable == 'Percent who walk to work','Variable'] = 'Percent_Walk'
    df.loc[df.Variable == 'Percent who use other means to commute to work','Variable'] = 'Percent_Other_Transport'
    df.loc[df.Variable == 'Percent who work from home','Variable'] = 'Percent_Home_Office'
    df.loc[df.Variable == 'Average travel time to work (mins)','Variable'] = 'Average_Commute_Time'
    df.loc[df.Variable == 'Percent with no vehicles available','Variable'] = 'Percent_0_Car'
    df.loc[df.Variable == 'Percent with 1 vehicle available','Variable'] = 'Percent_1_Car'
    df.loc[df.Variable == 'Percent with 2 or more vehicles available','Variable'] = 'Percent_2_Car'
    df.loc[df.Variable == 'Percent veterans','Variable'] = 'Percent_Veterans'
    df.loc[df.Variable == 'Percent foreign born','Variable'] = 'Percent_Foreign'
    df.loc[df.Variable == 'Percent speaking Spanish at home','Variable'] = 'Percent_Speak_Spanish'
    df.loc[df.Variable == 'Percent speaking other Indo-European languages at home','Variable'] = 'Percent_Speak_IndoEuropean'
    df.loc[df.Variable == 'Percent speaking other Asian and Pacific Island languages at home','Variable'] = 'Percent_Speak_Asian'
    
    df.replace(np.nan,0)
    
    return df

In [14]:
def add_socioeconomics_fields_to_row(df, row):
    fields = [
        'Median_Income', 'Average_Income', 'Percent_High_School', 'Percent_Bachelors', 'Average_Household_Size', 
        'Percent_In_Poverty', 'Percent_16_Over_Working', 'Percent_Employed', 'Percent_Disabled', 'Percent_Health_Coverage',
        'Percent_16_Over_Commuters', 'Percent_Drive_Alone', 'Percent_Drive_Carpool', 'Percent_Public_Transport', 'Percent_Walk',
        'Percent_Other_Transport', 'Percent_Home_Office', 'Average_Commute_Time', 'Percent_0_Car', 'Percent_1_Car', 
        'Percent_2_Car', 'Percent_Veterans', 'Percent_Foreign', 'Percent_Speak_Spanish', 'Percent_Speak_IndoEuropean',
        'Percent_Speak_Asian'
    ]    
    
    return add_fields(df, fields, row)

In [15]:
def get_housing(html):
    df = get_table_df(html, "Housing Characteristics")
    
    if(df is None):
        return None
        
    df.loc[df.Variable == 'Total number of housing units','Variable'] = 'Housing_Units_Total'
    df.loc[df.Variable == 'Owner-occupied housing units','Variable'] = 'Housing_Units_Owner'
    df.loc[df.Variable == 'Renter-occupied housing units','Variable'] = 'Housing_Units_Renter'
    df.loc[df.Variable == 'Single-family detached units','Variable'] = 'Housing_Units_Single_Family'
    df.loc[df.Variable == 'Home ownership rate','Variable'] = 'Ownership_Rate'
    df.loc[df.Variable == 'Vacancy rate','Variable'] = 'Vacancy_Rate'
    df.loc[df.Variable == 'Percent with a broadband internet subscription','Variable'] = 'Percent_Broadband'
    df.loc[df.Variable == 'Median owner-occupied housing unit value','Variable'] = 'Housing_Units_Median_Value'
    df.loc[df.Variable == 'Average housing value','Variable'] = 'Average_House_Value'
    df.loc[df.Variable == 'Median selected monthly owner cost','Variable'] = 'Median_Owner_Monthly_Cost'
    df.loc[df.Variable == 'Average monthly owner cost (with a mortgage)','Variable'] = 'Avg_Owner_Monthly_Cost'
    df.loc[df.Variable == 'Median rent (housing)','Variable'] = 'Median_Rent'
    df.loc[df.Variable == 'Average rent (housing)','Variable'] = 'Average_Rent'
    df.loc[df.Variable == 'Median year structure built','Variable'] = 'Median_Year_Built'
    
    df.replace(np.nan,0)
    
    return df

In [16]:
def add_housing_fields_to_row(df, row):
    fields = [
        'Housing_Units_Total', 'Housing_Units_Owner', 'Housing_Units_Renter', 'Housing_Units_Single_Family', 'Ownership_Rate', 
        'Vacancy_Rate', 'Percent_Broadband', 'Housing_Units_Median_Value', 'Average_House_Value', 'Median_Owner_Monthly_Cost', 
        'Avg_Owner_Monthly_Cost', 'Median_Rent', 'Average_Rent', 'Median_Year_Built'
    ]    
    
    return add_fields(df, fields, row)

In [17]:
def get_permits(html):
    df = get_table_df(html, "Building Permits")
    
    if(df is None):
        return None
        
    df.loc[df.Variable == '1 Unit Buildings','Variable'] = '1_Unit_Buildings'
    df.loc[df.Variable == '1 Unit Building Units','Variable'] = '1_Unit_Buildings_Units'
    df.loc[df.Variable == '1 Unit Building Value ($1,000)','Variable'] = '1_Unit_Buildings_Value'
    df.loc[df.Variable == '2 Unit Buildings','Variable'] = '2_Unit_Buildings'
    df.loc[df.Variable == '2 Unit Building Units','Variable'] = '2_Unit_Buildings_Units'
    df.loc[df.Variable == '2 Unit Building Value ($1,000)','Variable'] = '2_Unit_Buildings_Value'
    df.loc[df.Variable == '3-4 Unit Buildings','Variable'] = '3_4_Unit_Buildings'
    df.loc[df.Variable == '3-4 Unit Building Units','Variable'] = '3_4_Unit_Buildings_Units'
    df.loc[df.Variable == '3-4 Unit Building Value ($1,000)','Variable'] = '3_4_Unit_Buildings_Value'
    df.loc[df.Variable == '5+ Unit Buildings','Variable'] = '5_Over_Unit_Buildings'
    df.loc[df.Variable == '5+ Unit Building Units','Variable'] = '5_Over_Unit_Buildings_Units'
    df.loc[df.Variable == '5+ Unit Building Value ($1,000)','Variable'] = '5_Over_Unit_Buildings_Value'
    
    df.replace(np.nan,0)
    
    return df

In [18]:
def add_permits_fields_to_row(df, row):
    fields = [
        '1_Unit_Buildings', '1_Unit_Buildings_Units', '1_Unit_Buildings_Value', '2_Unit_Buildings', '2_Unit_Buildings_Units',
        '2_Unit_Buildings_Value', '3_4_Unit_Buildings', '3_4_Unit_Buildings_Units', '3_4_Unit_Buildings_Value', 
        '5_Over_Unit_Buildings', '5_Over_Unit_Buildings_Value', '5_Over_Unit_Buildings_Units'
    ]    
    
    return add_fields(df, fields, row)

In [19]:
def get_consumer(html):
    df = get_table_df(html, "Consumer Spending")
    
    if(df is None):
        return None
        
    df.loc[df.Variable == 'Total consumer expenditures per household','Variable'] = 'Consumer_Total_Expense'
    df.loc[df.Variable == 'Consumer expenditures per household on Alcoholic beverages','Variable'] = 'Consumer_Expense_Alcohol'
    df.loc[df.Variable == 'Consumer expenditures per household on Alcoholic beverages consumed at home','Variable'] = 'Consumer_Expense_Alcohol_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Beer consumed away from home','Variable'] = 'Consumer_Expense_Beer_Bar'
    df.loc[df.Variable == 'Consumer expenditures per household on Wine consumed away from home','Variable'] = 'Consumer_Expense_Wine_Bar'
    df.loc[df.Variable == 'Consumer expenditures per household on Apparel & services','Variable'] = 'Consumer_Expense_Clothes'
    df.loc[df.Variable == 'Consumer expenditures per household on Men\'s Apparel','Variable'] = 'Consumer_Expense_Mens_Clothes'
    df.loc[df.Variable == 'Consumer expenditures per household on Women\'s Apparel','Variable'] = 'Consumer_Expense_Womens_Clothes'
    df.loc[df.Variable == 'Consumer expenditures per household on Children\'s Apparel','Variable'] = 'Consumer_Expense_Childrens_Clothes' 
    df.loc[df.Variable == 'Consumer expenditures per household on Boys\' Apparel','Variable'] = 'Consumer_Expense_Boys_Clothes'
    df.loc[df.Variable == 'Consumer expenditures per household on Girls\' Apparel','Variable'] = 'Consumer_Expense_Girls_Clothes'
    df.loc[df.Variable == 'Consumer expenditures per household on Footware','Variable'] = 'Consumer_Expense_Footwear'
    df.loc[df.Variable == 'Consumer expenditures per household on Dining out (Food away from home)','Variable'] = 'Consumer_Expense_Dining'
    df.loc[df.Variable == 'Consumer expenditures per household on Dining out - Breakfast','Variable'] = 'Consumer_Expense_Dining_Breakfast'
    df.loc[df.Variable == 'Consumer expenditures per household on Dining out - Lunch','Variable'] = 'Consumer_Expense_Dining_Lunch'
    df.loc[df.Variable == 'Consumer expenditures per household on Dining out - Dinner','Variable'] = 'Consumer_Expense_Dining_Dinner'
    df.loc[df.Variable == 'Consumer expenditures per household on Education','Variable'] = 'Consumer_Expense_Education'
    df.loc[df.Variable == 'Consumer expenditures per household on Entertainment / Recreation','Variable'] = 'Consumer_Expense_Entertainment'
    df.loc[df.Variable == 'Consumer expenditures per household on Club membership fees','Variable'] = 'Consumer_Expense_Clubs'
    df.loc[df.Variable == 'Consumer expenditures per household on Dating services','Variable'] = 'Consumer_Expense_Dating'
    df.loc[df.Variable == 'Consumer expenditures per household on Pet food','Variable'] = 'Consumer_Expense_Pet_Food'
    df.loc[df.Variable == 'Consumer expenditures per household on Pet services','Variable'] = 'Consumer_Expense_Pet_Services'
    df.loc[df.Variable == 'Consumer expenditures per household on Food consumed at home','Variable'] = 'Consumer_Expense_Food_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Bakery products consumed at home','Variable'] = 'Consumer_Expense_Bakery_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Dairy products consumed at home','Variable'] = 'Consumer_Expense_Dairy_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Fruits and vegetables consumed at home','Variable'] = 'Consumer_Expense_Fruits_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Meat, poultry, fish, and eggs consumed at home','Variable'] = 'Consumer_Expense_Meat_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Nonalcoholic beverages consumed at home','Variable'] = 'Consumer_Expense_Nonalcohol_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Snacks and other food products consumed at home','Variable'] = 'Consumer_Expense_Snacks_Home'
    df.loc[df.Variable == 'Consumer expenditures per household on Health Care','Variable'] = 'Consumer_Expense_Healthcare'
    df.loc[df.Variable == 'Consumer expenditures per household on Medical services','Variable'] = 'Consumer_Expense_Mentalcare'
    df.loc[df.Variable == 'Consumer expenditures per household on Nonprescription drugs','Variable'] = 'Consumer_Expense_Drugcar'
    df.loc[df.Variable == 'Consumer expenditures per household on Household Services','Variable'] = 'Consumer_Expense_House_Services'
    df.loc[df.Variable == 'Consumer expenditures per household on Care for the elderly and handicapped','Variable'] = 'Consumer_Expense_Eldercare'
    df.loc[df.Variable == 'Consumer expenditures per household on Lawn and garden services','Variable'] = 'Consumer_Expense_Landscape'
    df.loc[df.Variable == 'Consumer expenditures per household on Housekeeping services','Variable'] = 'Consumer_Expense_Housekeeping'
    df.loc[df.Variable == 'Consumer expenditures per household on PC repair','Variable'] = 'Consumer_Expense_PC'
    df.loc[df.Variable == 'Consumer expenditures per household on Housing','Variable'] = 'Consumer_Expense_Housing'
    df.loc[df.Variable == 'Consumer expenditures per household on Home improvements','Variable'] = 'Consumer_Expense_Home_Improvements'
    df.loc[df.Variable == 'Consumer expenditures per household on Utilities, fuel, and public services','Variable'] = 'Consumer_Expense_Energy'
    df.loc[df.Variable == 'Consumer expenditures per household on Phone services','Variable'] = 'Consumer_Expense_Phone'
    df.loc[df.Variable == 'Consumer expenditures per household on Water','Variable'] = 'Consumer_Expense_Water'
    df.loc[df.Variable == 'Consumer expenditures per household on Life and other insurance','Variable'] = 'Consumer_Expense_Insurance'
    df.loc[df.Variable == 'Consumer expenditures per household on Pensions and social security','Variable'] = 'Consumer_Expense_Pensions'
    df.loc[df.Variable == 'Consumer expenditures per household on Personal Care Products and Services','Variable'] = 'Consumer_Expense_Personalcare'
    df.loc[df.Variable == 'Consumer expenditures per household on Hair care products','Variable'] = 'Consumer_Expense_Haircare'
    df.loc[df.Variable == 'Consumer expenditures per household on Personal care products','Variable'] = 'Consumer_Expense_Personalcare_Products'
    df.loc[df.Variable == 'Consumer expenditures per household on Local Transportation','Variable'] = 'Consumer_Expense_Transport'
    df.loc[df.Variable == 'Consumer expenditures per household on Gasoline','Variable'] = 'Consumer_Expense_Gas'
    df.loc[df.Variable == 'Consumer expenditures per household on Vehicle maintenance and repair','Variable'] = 'Consumer_Expense_Vehicle_Repair'
    df.loc[df.Variable == 'Consumer expenditures per household on Travel','Variable'] = 'Consumer_Expense_Travel'
    df.loc[df.Variable == 'Consumer expenditures per household on Airline fares','Variable'] = 'Consumer_Expense_Airfare'
    df.loc[df.Variable == 'Consumer expenditures per household on Auto/Truck rental on trips','Variable'] = 'Consumer_Expense_Auto_Rentals'
    df.loc[df.Variable == 'Consumer expenditures per household on Lodging on trips','Variable'] = 'Consumer_Expense_Travel_Lodging'
    df.loc[df.Variable == 'Consumer expenditures per household on Food and drink on trips','Variable'] = 'Consumer_Expense_Travel_Meals'
    df.loc[df.Variable == 'Consumer expenditures per household on Entertainment on trips','Variable'] = 'Consumer_Expense_Travel_Entertainment'
    
    df.replace(np.nan,0)
    
    return df

In [20]:
def add_consumer_fields_to_row(df, row):
    fields = [
        'Consumer_Total_Expense', 'Consumer_Expense_Alcohol', 'Consumer_Expense_Alcohol_Home', 'Consumer_Expense_Beer_Bar', 
        'Consumer_Expense_Wine_Bar', 'Consumer_Expense_Clothes', 'Consumer_Expense_Mens_Clothes', 'Consumer_Expense_Womens_Clothes', 
        'Consumer_Expense_Childrens_Clothes', 'Consumer_Expense_Boys_Clothes', 'Consumer_Expense_Girls_Clothes', 
        'Consumer_Expense_Footwear', 'Consumer_Expense_Dining', 'Consumer_Expense_Dining_Breakfast', 'Consumer_Expense_Dining_Lunch', 
        'Consumer_Expense_Dining_Dinner', 'Consumer_Expense_Education', 'Consumer_Expense_Entertainment', 'Consumer_Expense_Clubs', 
        'Consumer_Expense_Dating', 'Consumer_Expense_Pet_Food', 'Consumer_Expense_Pet_Services', 'Consumer_Expense_Food_Home', 
        'Consumer_Expense_Bakery_Home', 'Consumer_Expense_Dairy_Home', 'Consumer_Expense_Fruits_Home', 'Consumer_Expense_Meat_Home', 
        'Consumer_Expense_Nonalcohol_Home', 'Consumer_Expense_Snacks_Home', 'Consumer_Expense_Healthcare', 'Consumer_Expense_Mentalcare', 
        'Consumer_Expense_Drugcar', 'Consumer_Expense_House_Services', 'Consumer_Expense_Eldercare', 'Consumer_Expense_Landscape', 
        'Consumer_Expense_Housekeeping', 'Consumer_Expense_PC', 'Consumer_Expense_Housing', 'Consumer_Expense_Home_Improvements', 
        'Consumer_Expense_Energy', 'Consumer_Expense_Phone', 'Consumer_Expense_Water', 'Consumer_Expense_Insurance', 
        'Consumer_Expense_Pensions', 'Consumer_Expense_Personalcare', 'Consumer_Expense_Haircare', 
        'Consumer_Expense_Personalcare_Products', 'Consumer_Expense_Transport', 'Consumer_Expense_Gas', 
        'Consumer_Expense_Vehicle_Repair', 'Consumer_Expense_Travel', 'Consumer_Expense_Airfare', 'Consumer_Expense_Auto_Rentals', 
        'Consumer_Expense_Travel_Lodging', 'Consumer_Expense_Travel_Meals', 'Consumer_Expense_Travel_Entertainment'
    ]    
    
    return add_fields(df, fields, row)

In [21]:
def get_key_ratios(html):
    df = get_table_df(html, "Key Ratios")
    
    if(df is None):
        return None
        
    cols = {
        'Average employment per employer': 'Employee_Per_Employer', 
        'Average revenue per employer ($1,000)':'Revenue_Per_Employer', 
        'Average payroll per employee':'Avg_Payroll_Per_Employee', 
        'Population per employer':'Population_Per_Employer'
    }
    df.rename(columns=cols, inplace=True)
    
    # convert null values to zero
    df.loc[pd.isna(df.Employee_Per_Employer), "Employee_Per_Employer"] = '0'
    df.loc[pd.isna(df.Revenue_Per_Employer), "Revenue_Per_Employer"] = '0'
    df.loc[pd.isna(df.Avg_Payroll_Per_Employee), "Avg_Payroll_Per_Employee"] = '0'
    df.loc[pd.isna(df.Population_Per_Employer), "Population_Per_Employer"] = '0'

    # convert to int and strip chars ($,.)
    df.Employee_Per_Employer = convert_to_int(df.Employee_Per_Employer)
    df.Revenue_Per_Employer = convert_to_int(df.Revenue_Per_Employer)
    df.Avg_Payroll_Per_Employee = convert_to_int(df.Avg_Payroll_Per_Employee)
    df.Population_Per_Employer = convert_to_int(df.Population_Per_Employer)
    
    df = shorten_industries(df)
    
    return df

In [22]:
def add_ratio_fields_to_row(df, row):
    fields = ['Employee_Per_Employer', 'Revenue_Per_Employer', 'Avg_Payroll_Per_Employee', 'Population_Per_Employer']
    index_df = df.set_index('Industry')
    for i in industries:
        for f in fields:
            row[f'{i}_{f}'] = index_df[f][i]
    
    return row

In [25]:
# shorten all the industry names and format with underscores for column names
def shorten_industries(short_df):
    short_df.loc[short_df.Industry == 'All Sectors','Industry'] = 'All'
    short_df.loc[short_df.Industry == 'Accommodation and Food Services','Industry'] = 'Food_Services'
    short_df.loc[short_df.Industry == 'Administrative and Support and Waste Management and Remediation Services','Industry'] = 'Waste_Management'
    short_df.loc[short_df.Industry == 'Agriculture, Forestry, Fishing, and Hunting','Industry'] = 'Agriculture'
    short_df.loc[short_df.Industry == 'Arts, Entertainment, and Recreation','Industry'] = 'Arts'
    short_df.loc[short_df.Industry == 'Construction','Industry'] = 'Construction'
    short_df.loc[short_df.Industry == 'Educational Services','Industry'] = 'Education'
    short_df.loc[short_df.Industry == 'Finance & Insurance','Industry'] = 'Finance'
    short_df.loc[short_df.Industry == 'Health Care and Social Assistance','Industry'] = 'Healthcare'
    short_df.loc[short_df.Industry == 'Information','Industry'] = 'Information'
    short_df.loc[short_df.Industry == 'Management of Companies and Enterprises','Industry'] = 'Management'
    short_df.loc[short_df.Industry == 'Manufacturing','Industry'] = 'Manufacturing'
    short_df.loc[short_df.Industry == 'Mining and Oil and Gas Extraction','Industry'] = 'Mining'
    short_df.loc[short_df.Industry == 'Other Services','Industry'] = 'Other'
    short_df.loc[short_df.Industry == 'Professional, Scientific, and Technical Services','Industry'] = 'Technical'
    short_df.loc[short_df.Industry == 'Real Estate and Rental and Leasing','Industry'] = 'Real_Estate'
    short_df.loc[short_df.Industry == 'Retail Trade','Industry'] = 'Retail'
    short_df.loc[short_df.Industry == 'Transportation and Warehousing','Industry'] = 'Transportation'
    short_df.loc[short_df.Industry == 'Utilities','Industry'] = 'Utilities'
    short_df.loc[short_df.Industry == 'Wholesale Trade','Industry'] = 'Wholesale'
    
    return short_df

In [26]:
# GEO ID ######################

In [27]:
geo_data = pd.read_csv("geo_data.csv")
# ID
# GEOID
# PLACE_NAME
# PLACE_ID
# FIPS_CODE
# TYPE
# STATE_NAME
# STATE_ABBREVIATION

In [ ]:
# EXTRACT TABLE DATA ######################

In [882]:
# load locations from files (just MA for now)
loc_data = geo_data[geo_data.FIPS_CODE == 25]
print(loc_data.shape[0])

place_df = pd.DataFrame()
pos = 1
for index, row in loc_data.iterrows():
    print(pos)
    
    # update count
    pos += 1
    
    # get census data from file
    fips = row['FIPS_CODE']
    place = row['PLACE_NAME']
    state = row['STATE_ABBREVIATION']
    geoid = row['GEOID']
    
    # check if exists
    file_path = f'./census-html/{fips}-{state}-{place}-{geoid}.html'
    if not pathlib.Path(file_path).is_file():
        continue
    
    # get html
    f = open(file_path, "r")
    html = f.read()
    f.close()
    
    # start new row and add location information
    new_row = {
        'GeoId':geoid,
        'Place_Name':place,
        'State_Abbr':state,
        'State_Name':row['STATE_NAME'],
        'Type':row['TYPE']
    }
          
    # convert from long to wide - one entry for each place and append columns with the business totals   
    biz_df = get_business_totals(html)
    new_row = add_business_fields_to_row(biz_df, new_row)
    
    demo_df = get_demographics(html)
    new_row = add_demographics_fields_to_row(demo_df, new_row)
    
    socio_df = get_socioeconomics(html)
    new_row = add_socioeconomics_fields_to_row(socio_df, new_row)
        
    housing_df = get_housing(html)
    new_row = add_housing_fields_to_row(housing_df, new_row)
        
    permits_df = get_permits(html)
    new_row = add_permits_fields_to_row(permits_df, new_row)
        
    consumer_df = get_consumer(html)
    new_row = add_consumer_fields_to_row(consumer_df, new_row)        
    
    ratios_df = get_key_ratios(html)
    new_row = add_ratio_fields_to_row(ratios_df, new_row)    
    
    # combine new df and existing df
    new_row = pd.DataFrame(new_row, index=[0])
    place_df = pd.concat([new_row,place_df.loc[:]]).reset_index(drop=True)
    
print(place_df)

289
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276


In [ ]:
# FEATURE GENERATION ######################

In [113]:
# average_income per revenue (high is a likely a rich enclave)
ipr = place_df.Average_Income / place_df.All_Revenue
ipr = ipr.replace([np.inf, -np.inf], 0)   
place_df["Income_Per_Revenue"] = ipr

# number of industries supported (higher is more active)
revenue_df = place_df[[
    'All_Revenue', 'Food_Services_Revenue', 'Waste_Management_Revenue', 'Agriculture_Revenue', 'Arts_Revenue', 
    'Construction_Revenue', 'Education_Revenue', 'Finance_Revenue', 'Healthcare_Revenue', 'Information_Revenue', 
    'Management_Revenue', 'Manufacturing_Revenue', 'Mining_Revenue', 'Other_Revenue', 'Technical_Revenue', 
    'Real_Estate_Revenue', 'Retail_Revenue', 'Transportation_Revenue', 'Utilities_Revenue', 'Wholesale_Revenue'
]]
ind_sum = (revenue_df > 0).astype(int).sum(axis=1)
place_df["Industry_Count"] = ind_sum

# revenue per resident (high quality of living?)
rpp = place_df.All_Revenue / place_df.Population
rpp = rpp.replace([np.inf, -np.inf], 0)   
place_df["Revenue_Per_Person"] = rpp

# profit per person
ppp = place_df.Profit / place_df.Population
ppp = ppp.replace([np.inf, -np.inf], 0)   
place_df["Profit_Per_Person"] = ppp

# performance
perf_bools = (place_df.Population * 45 > place_df.All_Revenue)
perf = []
for b in perf_bools:
    p = "Below" if b else "Above"
    perf.append(p)
place_df["Performance"] = perf

# pop size 
place_df["Population_Range"] = pd.cut(place_df.Population, bins=[0, 5000, 10000, 20000, 75000, 125000, 1000000], 
       labels=["XSmall", "Small", "Medium", "Large", "XLarge", "XXLArge"])

# lat and long would help identify neighbors

# 
# needs fixing because it's zero
# All_Employee_Per_Employer
# All_Revenue_Per_Employer
# All_Avg_Payroll_Per_Employee
# All_Population_Per_Employer
#
# remove 'Town' from place name but fix places like edgar town and province town

In [ ]:
# SAVE DATASET TO FILE

In [28]:
#place_df.to_csv(".\places_data.csv")
#place_df = pd.read_csv("places_data.csv")